In [ ]:
# load libraries and scripts
import JSON
using Plots
using HDF5
using Trapz
using Statistics
include("../scripts/tools.jl")
include("../scripts/new_get_grid.jl")
include("../scripts/get_params.jl")
include("../scripts/stable.jl")
include("../scripts/dynamics.jl")

In [ ]:
#load borders
border = import_border("../data/border.json")
border = border
# create the lattice grid
dx = 20 
Nx, Ny, xrange, yrange, isinside, isborder, n = get_grid(border, Float64(dx))
isgrid = isinside .| isborder
# define x and y coordinate for plotting
x = repeat(reshape(xrange,1,Nx), Ny, 1)
y = repeat(reshape(yrange,Ny,1), 1, Nx);

In [ ]:
# compute or load grid parameters
recompute_param = true
if(recompute_param || isfile("../numerics/grid_params_" * string(dx) * ".h5"))
    #bx, by, p, m, d = get_params_diff(isinside, isborder, Float64(dx), yrange, xrange, "../data/pantagruel.h5",
    #    "../numerics/grid_params_" * string(dx) * ".h5", Niter=5000, dmax = 10.0, patch=0.002, bmin=0.01)
    bx, by, p, m, d = get_params(isinside, isborder, n, Float64(dx), yrange, xrange, "../data/pantagruel.h5",
        "../numerics/grid_params_" * string(dx) * ".h5", bmin=0.1, sigma=0.1)
else
    bx, by, p, m, d = get_params(isinside, "../numerics/grid_params_" * string(dx) * ".h5")
end

In [ ]:
# if grid params exist then plot bx and by
if(isfile("../numerics/grid_params_" * string(dx) * ".h5"))
    bx, by, p, m, d = get_params(isinside, "../numerics/grid_params_" * string(dx) * ".h5")
    x_for_bx = repeat(reshape(xrange,1,Nx),Ny,1) .+ dx/2
    y_for_bx = repeat(reshape(yrange,Ny,1),1,Nx)
    x_for_by = repeat(reshape(xrange,1,Nx),Ny,1)
    y_for_by = repeat(reshape(yrange,Ny,1),1,Nx) .+ dx/2;
    scatter(x[isgrid], y[isgrid], markershape=:circle, markersize=2, markerstrokewidth=0.2, label="nodes")
    id = findall(bx .> 0)
    scatter!(x_for_bx[id], y_for_bx[id], markershape=:circle, markersize=2, markerstrokewidth=0.2, label="bx")
    id = findall(by .> 0)
    scatter!(x_for_by[id], y_for_by[id], markershape=:circle, markersize=2, markerstrokewidth=0.2, label="by")
    savefig("susceptance.pdf")
end

In [ ]:

contour(m,fill=true)
contour(d,fill=true)
minimum(bx[bx.>0])
#println(sum(p[isborder]))

In [ ]:
# compute, iteratively, the stable solution
#bx .= max.(bx, 5)
#by .= max.(by, 5)

#bx = 50*ones(Ny, Nx)
#by = 50*ones(Ny, Nx)
th, ths = compute_stable_sol(isinside, n, bx, by, p, Niter = 10000, interval = 1000);

In [ ]:
contour(th)
sum(p[isborder])

In [ ]:
m = max.(m , 10) 
d = max.(d , 10)
thetas, omegas = perform_dyn_sim(isinside, n, bx, by, p, m, d, th,Ndt = 1000, interval=100, dt =1E-7)

In [ ]:
contour(d./m, fill=true)


In [ ]:
# save the result
fid = h5open("../numerics/stable_" * string(dx) * ".h5", "w")
write(fid, "th", th)
close(fid)

In [ ]:
# plot the result
@gif for i in 1:size(omegas,3)
    #hm_plot(isinside .| isborder, xrange, yrange, ths[:,:,i])
    println(maximum(abs.(omegas[:,:,i])))
    #ctr_plot(isinside .| isborder, omegas[:,:,i])
    #hm_plot(isinside .| isborder, ths[:,:,i], xlim=(90.0,110.0),ylim=(45.,60.))
end


In [ ]:
coord = [argmax(abs.(th[:,:]))[1], argmax(abs.(th[:,:]))[2]]

In [ ]:
bx[coord[1]+3:-1:coord[1]-3,coord[2]-3:coord[2]+3]

In [ ]:
by[coord[1]+3:-1:coord[1]-3,coord[2]-3:coord[2]+3]

In [ ]:
p[coord[1]+3:-1:coord[1]-3,coord[2]-3:coord[2]+3]

In [ ]:
th[coord[1]+3:-1:coord[1]-3,coord[2]-3:coord[2]+3]

In [ ]:
m[coord[1]+3:-1:coord[1]-3,coord[2]-3:coord[2]+3]

In [ ]:
isgrid[coord[1]+3:-1:coord[1]-3,coord[2]-3:coord[2]+3]

In [ ]:
isinside[coord[1]+3:-1:coord[1]-3,coord[2]-3:coord[2]+3]

In [ ]:
isborder[coord[1]+5:-1:coord[1]-3,coord[2]-5:coord[2]+3]

In [ ]:
for k in 1:size(n,1)
    
    i = Int64(n[k,1])
    j = Int64(n[k,2])
    if( (coord[1]-2 <= i) && (i <= coord[1]+2) && (coord[2]-2 <= j) && (j <= coord[2]+2) )       
        nx = n[k,4]
        ny = n[k,3]
        bij = (1 + ny) * by[i-1, j] + (1 - ny) * by[i, j] +
            (1 + nx) * bx[i, j-1] + (1 - nx) * bx[i, j]

        thij = (
            (1 + ny) * by[i-1, j] * th[i-1, j] +
            (1 - ny) * by[i, j] * th[i+1, j] + 
            (1 + nx) * bx[i, j-1] * th[i, j-1] +
            (1 - nx) * bx[i, j] * th[i,j+1] +
            dx^2 * p[i, j]
            ) / bij
        println([i j nx ny bij thij])
    end
end